<a href="https://colab.research.google.com/github/pvidya11/Deep_Learning/blob/main/Projects/05_RNN(LSTM)_NLP/Faker_News_Classifier_Using_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#train_file = "train.csv"
#test_file = "test.csv"
import pandas as pd  
train_df = pd.read_csv( "train.csv")
test_df = pd.read_csv( "test.csv")

In [3]:
train_df.head()

NameError: ignored

In [20]:
test_df.head()

,id,title,author,text
0,20800,"Specter of Trump Loosens Tongues, if Not Purse...",David Streitfeld,"PALO ALTO, Calif. — After years of scorning..."
1,20801,Russian warships ready to strike terrorists ne...,NaN,Russian warships ready to strike terrorists ne...
2,20802,#NoDAPL: Native American Leaders Vow to Stay A...,Common Dreams,Videos #NoDAPL: Native American Leaders Vow to...
3,20803,"Tim Tebow Will Attempt Another Comeback, This ...",Daniel Victor,"If at first you don’t succeed, try a different..."
4,20804,Keiser Report: Meme Wars (E995),Truth Broadcast Network,42 mins ago 1 Views 0 Comments 0 Likes 'For th...


In [19]:
train_df.shape, test_df.shape

((20800, 5), (5200, 4))

In [21]:
## drop NAN values
train_df = train_df.dropna()

In [22]:
train_df.shape, test_df.shape

((18285, 5), (5200, 4))

In [77]:
## Get the Independent Features
x_train = train_df.drop('label',axis=1)
x_test  = test_df

In [24]:
## Get the Dependent Features
y_train = train_df['label']

In [26]:
x_train.shape, y_train.shape

((18285, 4), (18285,))

In [27]:
import tensorflow as tf

In [28]:
tf.__version__

'2.4.1'

In [29]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [31]:
# Vocabulary size
voc_size = 5000

<h><b>Onehot Representation</h></b>

In [33]:
messages = x_train.copy()

In [34]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [35]:
messages.reset_index(inplace=True)

In [37]:
import nltk
import re
from nltk.corpus import stopwords

In [38]:
#Stop Words: A stop word is a commonly used word (such as “the”, “a”, “an”, “in”) 
# that a search engine has been programmed to ignore,
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [43]:
len(stopwords.words('english'))

179

In [46]:
# Dataset preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
  #print(i)
  review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
  review = review.lower()
  review = review.split()

  review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
  review = ' '.join(review)
  corpus.append(review)

In [ ]:
# processed titles
corpus

In [ ]:
onehot_repr = [one_hot(words, voc_size) for words in corpus]
onehot_repr

<h><b>Embedding Representation</b><h>

In [55]:
sent_len = 20
embedded_docs = pad_sequences(onehot_repr, padding='pre', maxlen = sent_len)
embedded_docs

array([[   0,    0,    0, ...,  729, 3441,  816],
       [   0,    0,    0, ..., 3808, 2216, 4094],
       [   0,    0,    0, ...,  557, 2517, 1919],
       ...,
       [   0,    0,    0, ..., 3561, 2572, 3823],
       [   0,    0,    0, ..., 3946,  131,  366],
       [   0,    0,    0, ..., 3650, 2968, 2219]], dtype=int32)

In [56]:
len(embedded_docs)

18285

In [57]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 2472,
       4174, 2455,  277, 3501, 4312, 3835,  729, 3441,  816], dtype=int32)

In [60]:
embedding_vector_features = 40
model = Sequential()
model.add(Embedding(voc_size, embedding_vector_features, input_length = sent_len))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 20, 40)            200000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               56400     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [62]:
len(embedded_docs),y_train.shape

(18285, (18285,))

In [63]:
import numpy as np
x_final = np.array(embedded_docs)
y_final = np.array(y_train)

In [64]:
x_final.shape,y_final.shape

((18285, 20), (18285,))

In [65]:
from sklearn.model_selection import train_test_split
x_train_1, x_test_1, y_train_1, y_test_1 = train_test_split(x_final, y_final, test_size=0.30, random_state=42)

<h><b>Model Training</b></h>

In [68]:
# finally training
#model.fit(x_train_1, y_train_1, validation_data=(x_test, y_test), epoch=10, batch_size=64)
model.fit(x_final, y_final, validation_split=0.30, epochs=10, batch_size=64)

Epoch 1/10
200/200 [==============================] - 10s 38ms/step - loss: 0.4672 - accuracy: 0.7601 - val_loss: 0.1962 - val_accuracy: 0.9169
Epoch 2/10
200/200 [==============================] - 7s 34ms/step - loss: 0.1361 - accuracy: 0.9439 - val_loss: 0.2026 - val_accuracy: 0.9112
Epoch 3/10
200/200 [==============================] - 7s 34ms/step - loss: 0.1018 - accuracy: 0.9642 - val_loss: 0.2123 - val_accuracy: 0.9171
Epoch 4/10
200/200 [==============================] - 7s 34ms/step - loss: 0.0716 - accuracy: 0.9731 - val_loss: 0.2360 - val_accuracy: 0.9132
Epoch 5/10
200/200 [==============================] - 7s 34ms/step - loss: 0.0543 - accuracy: 0.9830 - val_loss: 0.2757 - val_accuracy: 0.9107
Epoch 6/10
200/200 [==============================] - 7s 35ms/step - loss: 0.0437 - accuracy: 0.9860 - val_loss: 0.3216 - val_accuracy: 0.9085
Epoch 7/10
200/200 [==============================] - 7s 35ms/step - loss: 0.0301 - accuracy: 0.9910 - val_loss: 0.3784 - val_accuracy: 0.909

<h><b>Adding Dropout</b></h>

In [73]:
from tensorflow.keras.layers import Dropout
# Creating model
embedding_vector_features = 40
model = Sequential()
model.add(Embedding(voc_size, embedding_vector_features, input_length = sent_len))
model.add(Dropout(0.5))
model.add(LSTM(100))
model.add(Dropout(0.5))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [74]:

model.fit(x_final, y_final, validation_split=0.30, epochs=10, batch_size=64)

Epoch 1/10
200/200 [==============================] - 10s 38ms/step - loss: 0.4961 - accuracy: 0.7511 - val_loss: 0.2074 - val_accuracy: 0.9054
Epoch 2/10
200/200 [==============================] - 7s 35ms/step - loss: 0.1597 - accuracy: 0.9326 - val_loss: 0.1994 - val_accuracy: 0.9143
Epoch 3/10
200/200 [==============================] - 7s 35ms/step - loss: 0.1234 - accuracy: 0.9518 - val_loss: 0.1950 - val_accuracy: 0.9182
Epoch 4/10
200/200 [==============================] - 7s 35ms/step - loss: 0.0989 - accuracy: 0.9624 - val_loss: 0.2076 - val_accuracy: 0.9163
Epoch 5/10
200/200 [==============================] - 7s 35ms/step - loss: 0.0807 - accuracy: 0.9730 - val_loss: 0.2364 - val_accuracy: 0.9140
Epoch 6/10
200/200 [==============================] - 7s 36ms/step - loss: 0.0665 - accuracy: 0.9770 - val_loss: 0.2394 - val_accuracy: 0.9200
Epoch 7/10
200/200 [==============================] - 7s 36ms/step - loss: 0.0548 - accuracy: 0.9808 - val_loss: 0.2612 - val_accuracy: 0.915

<h><b>Performans Metrics And Accuracy</h></b>

In [91]:
def pre_processing(df):
    messages = df.copy()
    messages.reset_index(inplace=True)
    ps = PorterStemmer()
    corpus = []
    for i in range(0, len(messages)):
        #print(messages['title'][i])
        try:
          review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
        except Exception as e:
          print(messages['title'][i])
        review = review.lower()
        review = review.split()
        review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
        review = ' '.join(review)
        corpus.append(review)

    onehot_repr = [one_hot(words, voc_size) for words in corpus]

    sent_len = 20
    embedded_docs = pad_sequences(onehot_repr, padding='pre', maxlen = sent_len)

    x_final = np.array(embedded_docs)
    return x_final

In [100]:
x_test = x_test.dropna()
x_test_final = pre_processing(x_test)
x_test_final

array([[   0,    0,    0, ..., 3561, 2572, 3823],
       [   0,    0,    0, ...,  522, 1399, 2630],
       [   0,    0,    0, ..., 3561, 2572, 3823],
       ...,
       [   0,    0,    0, ..., 3561, 2572, 3823],
       [   0,    0,    0, ..., 3561, 2572, 3823],
       [   0,    0,    0, ..., 3561, 2572, 3823]], dtype=int32)

In [101]:
len(x_test_final)

4575

In [98]:
y_pred = np.argmax(model.predict(x_test_final), axis=-1)

array([0, 0, 0, ..., 0, 0, 0])

In [99]:
set(y_pred)


{0}

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
confusion_metrix(y_test,y_pred)

In [ ]:
from sklearn.metrics import accuracy_score 
accuracy_score(y_test,y_pred)